In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
import cv2
import numpy as np
import time
import datetime
from keras.models import load_model

labels_dict={1:'smile', 0:'not smile'}
color_dict={0:(0,255,0),1:(0,0,255)}

fourcc = cv2.VideoWriter_fourcc(*'DIVX')

font = cv2.FONT_HERSHEY_SIMPLEX

face_cascade = cv2.CascadeClassifier('../data/haarcascade_frontalface_alt.xml')

if face_cascade.empty():
    raise IOError('Unable to load the face cascade classifier xml file')

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('../data/capture1/trainer/trainer.yml')

model = load_model('../data/capture1/093-0.1248.model')

record = False
face_filter = False 
user_detection = False
smile_detection = False

"""
# 비디오 재생
video_dir = "../data/mask/"
video_name = "girl.mp4"
cam=cv2.VideoCapture(video_dir + video_name)
"""

# 실시간 영상
cam=cv2.VideoCapture(0)

width = cam.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cam.get(cv2.CAP_PROP_FRAME_HEIGHT)

id = 0, 1, 2, 3
names = ['SY', 'JM', 'HS', 'BJ']

s_mask = cv2.imread('../data/mask/7.jpg')
a_mask = cv2.imread('../data/mask/8.jpg')

while True:
    ret,frame = cam.read()
    
    if not ret:
        print("영상 끝 - 종료")
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_rects = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5)
    
    if face_filter:
        info = "Face Filter ON"
        if user_detection:
            info = "Face Filter & User Detection ON"
            if smile_detection:
                info = "All Filter ON"
        elif smile_detection:
            info = "Face Filter & Smile Detection ON"
    elif user_detection:
        info = "User Detection ON"
        if smile_detection:
                info = "User Detection & Smile Detection ON"
    elif smile_detection:
        info = "Smile Detection ON"
    else:
        info = "All Filter OFF"            
    
    cv2.putText(frame, info, (5,50), font, 2, (188, 237, 75),2)
    
    for (x, y, w, h) in face_rects:        
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        # 미소 인식
        if smile_detection:
            if(labels_dict[label] == 'smile'):
                cv2.putText(
                frame, "{}: {:.2f}%".format(labels_dict[label], np.max(result) * 100),
                (x, y-10),
                font,0.6,(255,0,216),2)
            else:
                cv2.putText(
                frame, "{}: {:.2f}%".format(labels_dict[label], np.max(result) * 100),
                (x, y-10),
                font,0.6,(216,255,0),2)
            
        # 사용자 인식
        if user_detection:
            id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
        
            if (confidence < 100):
                id = names[id]
                confidence = "  {0}%".format(round(100 - confidence))
            else:
                id = "unknown"
                confidence = "  {0}%".format(round(100 - confidence))

            cv2.putText(frame, str(id), (x+90, y-45), font, 1, (0,216,255), 3)
            cv2.putText(frame, str(confidence), (x+80,y-30), font, 0.5, (0,216,255), 1)            
        
        # 미소 인식 필터
        if face_filter: 
            try:    
                if(labels_dict[label] == 'smile'):
                    if h > 0 and w > 0:
                        x = int(x -w*0.1)
                        y = int(y -h*0.05)
                        w = int(1.2 * w)
                        h = int(1.2 * h)

                        frame_roi = frame[y:y + h, x:x + w]
                        face_mask_small = cv2.resize(s_mask, (w, h), interpolation=cv2.INTER_AREA)
                        gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                        ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_BINARY_INV)

                    mask_inv = cv2.bitwise_not(mask)
                    masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                    masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                    frame[y:y + h, x:x + w] = cv2.add(masked_face, masked_frame)

                else:
                    if h > 0 and w > 0:
                        x = int(x -w*0.1)
                        y = int(y -h*0.05)
                        w = int(1.2 * w)
                        h = int(0.8 * h)

                        frame_roi = frame[y:y + h, x:x + w]
                        face_mask_small = cv2.resize(a_mask, (w, h), interpolation=cv2.INTER_AREA)
                        gray_mask = cv2.cvtColor(face_mask_small, cv2.COLOR_BGR2GRAY)
                        ret, mask = cv2.threshold(gray_mask, 240, 255, cv2.THRESH_BINARY_INV)

                    mask_inv = cv2.bitwise_not(mask)
                    masked_face = cv2.bitwise_and(face_mask_small, face_mask_small, mask=mask)
                    masked_frame = cv2.bitwise_and(frame_roi, frame_roi, mask=mask_inv)
                    frame[y:y + h, x:x + w] = cv2.add(masked_face, masked_frame)
            except Exception:
                pass

    key = cv2.waitKey(1)
    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    
    cv2.imshow('Project', frame)
    
    if key == 27: # ESC
        print("ESC - 종료")
        break
    elif key == 105: # i
        face_filter = not face_filter
    elif key == 117: # u
        user_detection = not user_detection
    elif key == 116: # t
        smile_detection = not smile_detection
    elif key == 99: # c
        print("캡쳐")
        cv2.imwrite("../data/capture1/" + str(now) + ".png", frame)
    elif key == 115: # s
        print("녹화 시작")
        record = True
        out = cv2.VideoWriter("../data/capture1/" + str(now) + ".avi", fourcc, 30, (int(width), int(height)))
    elif key == 122: # z
        print("녹화 중지")
        record = False
        out.release()
        
    if record == True:
        out.write(frame)

cam.release()
cv2.destroyAllWindows()

ESC - 종료
